In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [2]:
from datasets import load_dataset
import tensorflow as tf
from transformers import BertTokenizer,TFBertForSequenceClassification

In [3]:
dataset=load_dataset('ai4bharat/IndicSentiment',name='translation-mr')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/980 [00:00<?, ?B/s]

IndicSentiment.py:   0%|          | 0.00/2.51k [00:00<?, ?B/s]

The repository for ai4bharat/IndicSentiment contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ai4bharat/IndicSentiment.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


mr.json:   0%|          | 0.00/183k [00:00<?, ?B/s]

mr.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
training_texts=dataset['test']['INDIC REVIEW']
training_labels=dataset['test']['LABEL']
validation_texts=dataset['validation']['INDIC REVIEW']
validation_labels=dataset['validation']['LABEL']

In [5]:
training_labels=[1 if label=='Positive' else 0 for label in training_labels]
validation_labels=[1 if label=='Positive' else 0 for label in validation_labels]

In [6]:
tokenizer=BertTokenizer.from_pretrained('bert-base-multilingual-cased')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [7]:
encoded_training_texts=tokenizer(training_texts,padding=True,truncation=True,return_tensors='tf')
encoded_validation_texts=tokenizer(validation_texts,padding=True,truncation=True,return_tensors='tf')


In [8]:
train_loader=tf.data.Dataset.from_tensor_slices((dict(encoded_training_texts),training_labels)).batch(8)
val_loader=tf.data.Dataset.from_tensor_slices((dict(encoded_validation_texts),validation_labels)).batch(8)

In [9]:
model=TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
model.fit(train_loader,epochs=3,validation_data=val_loader,verbose=1)

Epoch 1/3
125/125 [==============================] - 125s 536ms/step - loss: 0.6115 - accuracy: 0.6680 - val_loss: 0.5669 - val_accuracy: 0.7372
Epoch 2/3
125/125 [==============================] - 64s 510ms/step - loss: 0.4490 - accuracy: 0.8030 - val_loss: 0.4060 - val_accuracy: 0.8397
Epoch 3/3
125/125 [==============================] - 64s 511ms/step - loss: 0.2775 - accuracy: 0.8950 - val_loss: 0.5157 - val_accuracy: 0.8333
